In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import svm

In [382]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [383]:
downloaded = drive.CreateFile({'id':'1wd4DXf4nb9dWwieEc3LMkfHfc5V51bha'}) 
downloaded.GetContentFile('train.csv') 
downloaded = drive.CreateFile({'id':'1UUPUpFC5hiPO9066Cz7tfUzlmOK5SsES'}) 
downloaded.GetContentFile('test.csv')

In [384]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [385]:
test_df["Sex"]=pd.get_dummies(test_df["Sex"])
train_df["Sex"]=pd.get_dummies(train_df["Sex"])

In [386]:
train_df['Age'].fillna(train_df['Age'].median(), inplace = True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

train_df['Fare'].fillna(train_df['Fare'].median(), inplace = True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace = True)

In [387]:
train_df['NewAge'] = pd.cut(train_df['Age'], bins=[0,15,30,60,120], labels=['c','t','a','e'])
test_df['NewAge'] = pd.cut(test_df['Age'], bins=[0,12,20,40,120], labels=['c','t','a','e'])

In [389]:
train_df['NewFare'] = pd.cut(train_df['Fare'], bins=[0,15,30,60,120], labels=['lf','mf','af','hf'])
test_df['NewFare'] = pd.cut(test_df['Fare'], bins=[0,15,30,60,120], labels=['lf','mf','af','hf'])


In [390]:
train_df = pd.get_dummies(train_df, columns = ["NewAge","Embarked"],prefix=["Age_type","Em_type"])
test_df = pd.get_dummies(test_df, columns = ["NewAge","Embarked"],prefix=["Age_type","Em_type"])

In [392]:
train_df = pd.get_dummies(train_df, columns = ["NewFare"],prefix=["Fare_type"])
test_df = pd.get_dummies(test_df, columns = ["NewFare"],prefix=["Fare_type"])

In [393]:
train_df = train_df.drop(['Name','PassengerId','Cabin','Ticket'], axis=1)
test_df = test_df.drop(['Name','PassengerId','Cabin','Ticket'], axis=1)

In [394]:
train_df = train_df.drop(['Fare','Age'], axis=1)
test_df = test_df.drop(['Fare','Age'], axis=1)

In [395]:
f = train_df.drop("Survived",axis=1)
t = train_df["Survived"]
X_train,X_test,y_train,y_test = train_test_split(f,t,test_size=0.3,random_state=1)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((623, 15), (268, 15), (623,), (268,))

In [396]:
clf = svm.SVC(kernel='linear')   
clf.fit(X_train, y_train.ravel());
accuracy_score = cross_val_score(clf, X_train, y_train.ravel(), cv=5, scoring='accuracy').mean()
print("Accuracy of Linear Kernel is", accuracy_score)

Accuracy of Linear Kernel is 0.812232258064516


In [397]:
clf = svm.SVC(kernel='poly', degree=2)
clf.fit(X_train, y_train.ravel());
accuracy_score = cross_val_score(clf, X_train, y_train.ravel(), cv=5, scoring='accuracy').mean()
print("Accuracy of quadratic Kernel ", accuracy_score)

Accuracy of quadratic Kernel  0.820232258064516


In [398]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train.ravel());

accuracy_score = cross_val_score(clf, X_train, y_train.ravel(), cv=5, scoring='accuracy').mean()
print("Accuracy of RBF Kernel is ", accuracy_score)

Accuracy of RBF Kernel is  0.8443096774193549
